- SAD
- MSE
- structural similarity
- F1


In [132]:
from skimage import metrics
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage.metrics import adapted_rand_error, variation_of_information
from sklearn.metrics import confusion_matrix  

In [133]:
def sad(imageA, imageB):
   
    # sum of the abs difference between the two images;
    err = np.sum(np.abs(imageA.astype("float") - imageB.astype("float")))
    err /= float(imageA.shape[0] * imageA.shape[1])

    return err

In [134]:
def mse(imageA, imageB):
    
    # sum of the squared difference between the two images;
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])

    return err


In [135]:
def compute_iou(y_pred, y_true):
    # ytrue, ypred is a flatten vector
    y_pred = y_pred.flatten()
    y_true = y_true.flatten()
    current = confusion_matrix(y_true, y_pred, labels=[0, 1])
    # compute mean iou
    intersection = np.diag(current)
    ground_truth_set = current.sum(axis=1)
    predicted_set = current.sum(axis=0)
    union = ground_truth_set + predicted_set - intersection
    IoU = intersection / union.astype(np.float32)
    return np.mean(IoU)

In [143]:
def compare_images(imageA, imageB, title):

    m = mse(imageA, imageB)
    s = metrics.structural_similarity(imageA, imageB)
    d = sad(imageA, imageB)
    
    error, precision, recall = adapted_rand_error(imageA, imageB)
    splits, merges = variation_of_information(imageA, imageB)

    f1 = 2 * precision * recall / (precision + recall)
    iou = compute_iou(imageA, imageB)

#     fig = plt.figure(title)
#     plt.suptitle("MSE: %.2f, SAD: %.2f, SSIM: %.2f, F1: %.2f, IoU: %.2f" % (m, d, s, f1, iou))

#     ax = fig.add_subplot(1, 2, 1)
#     plt.imshow(imageA, cmap = plt.cm.gray)
#     plt.axis("off")

#     ax = fig.add_subplot(1, 2, 2)
#     plt.imshow(imageB, cmap = plt.cm.gray)
#     plt.axis("off")

#     plt.show()

#     print('mse: ', m)
#     print('sad: ', d)
#     print('structural similarity: ', s)

#     print('f1: ', f1)
#     print('iou: ', iou)
    
#     print('precision: ', precision)
#     print('recall: ', recall)

    print('mse, sad, ssim, f1')

    return m, d, s, f1

In [144]:
# impath1 = 'test/test.png'
# impath2 = 'test/test_binarized.png'

impath1 = 'objective_test_upd/original_masks/boi_jeans_shirt_107-removebg-preview.png'
impath2 = 'objective_test_upd/our_masks/boi_jeans_shirt_107-removebg-preview.png'
impath3 = 'objective_test_upd/removebg_masks/boi_jeans_shirt_107-removebg-preview.png'

In [145]:
img1 = cv2.imread(impath1)
img2 = cv2.imread(impath2)
img3 = cv2.imread(impath3)

img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)

In [147]:
compare_images(img1, img3, "Original vs. Removebg")

mse, sad, ssim, f1


/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


(0.0, 0.0, 1.0, 1.0)

In [146]:
compare_images(img1, img2, "Original vs. Ours")

mse, sad, ssim, f1


(356.3240610328638, 3.10431338028169, 0.946530726918942, 0.6866285816059222)

In [148]:
compare_images(img3, img2, "Removebg vs. Ours")

mse, sad, ssim, f1


(356.3240610328638, 3.10431338028169, 0.946530726918942, 0.6866285816059222)

In [115]:
import glob

In [149]:
path1_gt = 'objective_test_upd/original_masks/'
path2_nano = 'objective_test_upd/our_masks/'
path3_rmbg = 'objective_test_upd/removebg_masks/'

In [150]:
gt_files = glob.glob(path1_gt+'*')
nano_files = glob.glob(path2_nano+'*')
rmbg_files = glob.glob(path3_rmbg+'*')

In [162]:
gt_files.sort()
nano_files.sort()
rmbg_files.sort()

In [176]:
len(gt_files), len(nano_files), len(rmbg_files)

(2721, 2720, 2716)

In [166]:
gt_files[1].split('/')[-1]

'boi_jeans_shirt_187-removebg-preview.png'

In [183]:
gt_files[437].split('/')[-1]

'bossy_girl_31-removebg-preview.png'

In [184]:
nano_files[437].split('/')[-1]

'bossy_girl_36-removebg-preview.png'

In [185]:
rmbg_files[437].split('/')[-1]

'bossy_girl_36-removebg-preview.png'

In [181]:
#check 

# for i in range (len(gt_files)):
#     if not((gt_files[i].split('/')[-1] == nano_files[i].split('/')[-1]) and (nano_files[i].split('/')[-1]==rmbg_files[i].split('/')[-1])):
#         print(i)

In [127]:
import time

In [177]:
len(gt_files[:430])

430

In [186]:
import numpy as np
import os
import cv2
def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [190]:
res = noisy('poisson', img3)

In [191]:
i

0

In [192]:
img3 = cv2.imread(rmbg_files[i]) # их 
    
res = noisy('gauss', img3)

In [196]:
res.shape

(320, 213, 3)

In [214]:

img1 = cv2.imread(gt_files[i]) # эталон
img2 = cv2.imread(nano_files[i]) # наша
img3 = cv2.imread(rmbg_files[i]) # их 

img3n = noisy('s&p', img3)

/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [215]:
img3n.shape

(320, 213, 3)

In [216]:
img1.shape

(320, 213, 3)

In [217]:
img1.max()

255

In [218]:
img3n.max()

255

In [219]:
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img3n = cv2.cvtColor(img3n, cv2.COLOR_BGR2GRAY)

In [220]:
mse_nano = []
mse_rmbg = []

sad_nano = []
sad_rmbg = []

ssim_nano = []
ssim_rmbg = []

f1_nano = []
f1_rmbg = []

start = time.monotonic()
for i in range(len(gt_files[:430])):
    
    img1 = cv2.imread(gt_files[i]) # эталон
    img2 = cv2.imread(nano_files[i]) # наша
    img3 = cv2.imread(rmbg_files[i]) # их 
    
    img3n = noisy('s&p', img3)

    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    img3n = cv2.cvtColor(img3n, cv2.COLOR_BGR2GRAY)
    
    res1 = compare_images(img1, img2, "Original vs. Nanosemantics")
    res2 = compare_images(img1, img3n, "Original vs. Removebg")
    
    mse_nano.append(res1[0])
    mse_rmbg.append(res2[0])

    sad_nano.append(res1[1])
    sad_rmbg.append(res2[1])

    ssim_nano.append(res1[2])
    ssim_rmbg.append(res2[2])

    f1_nano.append(res1[3])
    f1_rmbg.append(res2[3])

end = time.monotonic() - start

/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result eith

mse, sad, ssim, f1
mse, sad, ssim, f1
mse, sad, ssim, f1
mse, sad, ssim, f1


/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result eith

mse, sad, ssim, f1
mse, sad, ssim, f1
mse, sad, ssim, f1
mse, sad, ssim, f1


/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result eith

...

/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


mse, sad, ssim, f1
mse, sad, ssim, f1


/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


mse, sad, ssim, f1
mse, sad, ssim, f1
mse, sad, ssim, f1


/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [221]:
end

66.57081563601969

In [222]:
print('Меньше - лучше: ')
print(' ')
print('MSE nanosemanitcs: ', sum(mse_nano))
print('MSE remove.bg : ', sum(mse_rmbg))
print(' ')

print('SAD nanosemanitcs: ', sum(sad_nano))
print('SAD remove.bg : ', sum(sad_rmbg))
print(' ')
print(' ')
print(' ')

print('Больше - лучше: ')
print(' ')

print('SSim nanosemanitcs: ', sum(ssim_nano))
print('SSim remove.bg : ', sum(ssim_rmbg))
print(' ')

print('F1 nanosemanitcs: ', sum(f1_nano))
print('F1 remove.bg : ', sum(f1_rmbg))

Меньше - лучше: 
 
MSE nanosemanitcs:  151601.29354760208
MSE remove.bg :  29381.71046709286
 
SAD nanosemanitcs:  1392.5349860611989
SAD remove.bg :  207.9381868050952
 
 
 
Больше - лучше: 
 
SSim nanosemanitcs:  411.0068555949822
SSim remove.bg :  385.02421189606133
 
F1 nanosemanitcs:  348.325989530301
F1 remove.bg :  424.6913901438759
